In [ ]:
import os

os.chdir('Sidecar/code/NLS/moving/PINNs_withR')

gpu_list = [0, 1, 2, 3]
cuda_device = ','.join([str(i) for i in gpu_list])

print(f'Using GPU: {cuda_device}')

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import traceback
from collections import OrderedDict
from tqdm import tqdm, trange

import random

from tensorboardX import SummaryWriter

from scipy import integrate

sns.set_style("white")
torch.cuda.is_available()


from PINNs_para import parallel_train
from parameters import params

torch.set_default_dtype(torch.float64)

Using GPU: 0,1,2,3


# Parallel training

In [ ]:
import torch.multiprocessing as mp

num_processes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
            10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

param_vector = [50, 100, 200, 400]

print(param_vector)

result_matrix = np.zeros((len(param_vector), len(num_processes), 5))


i = 0

for width in tqdm(param_vector):
    
    shared_params = params()

    shared_params.width = width
    shared_params.epoches = 100000
    shared_params.n_inner_points = 128
    shared_params.n_bic_points = 128
    shared_params.x_l = -15.0
    shared_params.x_r = 15.0
    shared_params.batch_size = 2046

    if __name__ == '__main__':
        mp.set_start_method('spawn', force=True)

        manager = mp.Manager()
        results_queue = manager.Queue()

        num_tasks = num_processes
        num_gpus = len(gpu_list)

        processes = []

        for task_id in num_tasks:
            
            gpu_id = task_id % num_gpus

            print(f"Task {task_id} on GPU {gpu_list[gpu_id]}")
                
            p = mp.Process(target=parallel_train, args=(task_id, shared_params, results_queue, gpu_id))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()
    

        while not results_queue.empty():
            (idx, loss_vector) = results_queue.get()
            result_matrix[i, idx, :] = loss_vector
            
    i += 1

print(result_matrix)